In [8]:
from google.transliteration import transliterate_word
suggestions = transliterate_word('America', lang_code='bn')
print(suggestions)

['আমেরিকা', 'এমেরিকা', 'আমেৰিকা', 'আমেরিকে', 'অমেরিকা', 'আমেরিকাঃ']


# STEP1: eng_bn transiterate

In [19]:
most_freq = 1000

In [20]:
import nltk
nltk.download('averaged_perceptron_tagger')

stopwords = set(nltk.corpus.stopwords.words('english'))
pos_tags = nltk.pos_tag(stopwords)
pronouns = set([word.lower() for (word, tag) in pos_tags if tag in ['PRP', 'PRP$']])
pronouns.add('us'  )
print(pronouns)
prepositions = set([word.lower() for (word, tag) in pos_tags if tag == 'IN'])
print(prepositions)

{'himself', 'it', 'them', 'my', 'her', 'they', 'your', 'itself', 'yourself', 'herself', 'his', 'him', 'you', 'we', 'me', 'he', 'myself', 'ourselves', 'themselves', 'our', 'its', 'she', 'us', 'their'}
{'as', 'with', 'through', "weren't", 'from', 'at', 'between', 'below', 'before', 'because', "it's", 'for', 'than', 'about', 'after', 'in', 'above', 'on', 'if', 'over', 'isn', 'during', 'while', 'until', 'of', 'against', 'under', 'that', 'by', 'into'}


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [21]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
# from indic_transliteration import sanscript

# Read the unigram_freq.csv file
df = pd.read_csv('unigram_freq.csv')

# Remove stop words using NLTK
stop_words = set(stopwords.words('english'))
# ============= new add =========================

df = df[~df['word'].isin(stop_words)]
# ============= new add =========================
df = df[~df['word'].isin(pronouns)]
df = df[~df['word'].isin(prepositions)]

# Take 10000 most frequent words
df = df.nlargest(most_freq, 'count')


In [22]:
df.head(5)

,word,count
26,new,1551258643
32,home,1276852170
37,page,1082121730
40,search,1024093118
41,free,1014107316


In [23]:
df.shape

(1000, 2)

In [24]:
from google.transliteration import transliterate_word
# Define a function to transliterate a given string to Bengali script
def transliterate_to_bengali(text):
    suggestions = transliterate_word(text, lang_code='bn')
    # print("suggestions: ", suggestions)
    top_suggestion = suggestions[0]
    return top_suggestion

In [27]:
# save copy of df
df_copy = df.copy()


In [26]:
# split dataframe into 10 dataframes
import numpy as np
dfs = np.array_split(df, 10)

# assign dataframes to separate variables
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 = dfs

In [28]:
for i, df in enumerate(dfs):
    print("df_", i+1)
    print(df.head(2))
    df['bangla'] = df['word'].apply(transliterate_to_bengali)
    df = df.rename(columns={'word': 'english'})
    df.to_csv(f'df{i+1}.csv', index=False)


df_ 1
    word       count
26   new  1551258643
32  home  1276852170
df_ 2
         word      count
190   general  311757793
191  research  311538382
df_ 3
         word      count
301    county  227567373
302  american  227534978
df_ 4
        word      count
407  include  182579275
408  college  182545426
df_ 5
            word      count
508  application  152776595
509         cart  152155277
df_ 6
      word      count
613  write  126645151
614    war  126517399
df_ 7
     word      count
715  sure  110528740
716   faq  110323671
df_ 8
        word     count
817  hosting  97747750
818    rules  97658641
df_ 9
        word     count
917   beauty  87572240
918  manager  87441704
df_ 10
         word     count
1018     pics  79323162
1019  western  79255339


In [29]:
df_merge = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index=True)
df_merge.shape


(1000, 3)

In [30]:
df_merge.head(5)

,word,count,bangla
0,new,1551258643,নিউ
1,home,1276852170,হোম
2,page,1082121730,পেজ
3,search,1024093118,সার্চ
4,free,1014107316,ফ্রি


In [32]:
df_merge = df_merge.rename(columns={'word': 'english'})

In [33]:
# divide the 'count' column values by 1 million to convert to millions range
df_merge['count_millions'] = df_merge['count'] // 1000000
df_merge = df_merge[['english', 'bangla', 'count','count_millions']]

In [34]:
df_merge.to_csv('en_bn.csv', index=False)
